In [1]:
import os
import sys
import time
import json
from os.path import abspath, dirname
import pandas as pd
import numpy as np
import datetime

# current_dir = abspath(dirname(__file__))
current_dir = abspath(dirname(""))
project_path = abspath(dirname(dirname(current_dir)))
sys.path.append(project_path + "/TradeStreamEngine/StreamEngine")


import lookups

# Indicators aggregate

In [2]:
from flow import oiFundingflow
from synthesis import oiomnifier

data_binance_btcusdt_OI = json.load(open("data/binance_btcusdt_perpetual_OI.json"))
data_binance_btcusdt_funding = json.load(open("data/binance_btcusdt_perpetual_fundingRate.json"))
data_binance_btcusd_OI = json.load(open("data/binance_btcusd_perpetual_OI.json"))
data_binance_btcusd_funding = json.load(open("data/binance_btcusd_perpetual_fundingRate.json"))
data_okx_btcusdt_OI = json.load(open("data/okx_btcusdt_perpetual_OI.json"))
data_okx_btcusdt_funding = json.load(open("data/okx_btcusdt_perpetual_fundingRate.json"))
data_okx_btcusd_OI = json.load(open("data/okx_btcusd_perpetual_OI.json"))
data_okx_btcusd_funding = json.load(open("data/okx_btcusd_perpetual_fundingRate.json"))
data_bybit_btcusdt_oifunding = json.load(open("data/bybit_btcusdt_perpetual_fundingRate_OI.json"))

binance1o  = oiFundingflow('binance', 'btc_usdt', 'perpetual', 20, lookups.binance_OI_lookup, lookups.binance_funding_lookup)
binance2o  = oiFundingflow('binance', 'btc_usd', 'perpetual', 20, lookups.binance_OI_lookup, lookups.binance_funding_lookup)
okx1o  = oiFundingflow('okx', 'btc_usdt', 'perpetual', 20, lookups.okx_OI_lookup, lookups.okx_funding_lookup)
okx2o  = oiFundingflow('okx', 'btc_usd', 'perpetual', 20, lookups.okx_OI_lookup, lookups.okx_funding_lookup)
bybitOF  = oiFundingflow('bybit', 'btc_usdt', 'perpetual', 20, lookups.bybit_OI_funding_lookup)


for index, d in enumerate(data_binance_btcusdt_funding):
    d = json.dumps(d)
    binance1o.input_funding(d)
for index, d in enumerate(data_binance_btcusdt_OI):
    d = json.dumps(d)
    binance1o.input_oi(d)

for index, d in enumerate(data_binance_btcusd_funding):
    d = json.dumps(d)
    binance2o.input_funding(d)
for index, d in enumerate(data_binance_btcusd_OI):
    d = json.dumps(d)
    binance2o.input_oi(d)

for index, d in enumerate(data_okx_btcusdt_funding):
    d = json.dumps(d)
    okx1o.input_funding(d)
for index, d in enumerate(data_okx_btcusdt_OI):
    d = json.dumps(d)
    okx1o.input_oi(d)

for index, d in enumerate(data_okx_btcusd_funding):
    d = json.dumps(d)
    okx2o.input_funding(d)
for index, d in enumerate(data_okx_btcusd_OI):
    d = json.dumps(d)
    okx2o.input_oi(d)

for index, d in enumerate(data_bybit_btcusdt_oifunding):
    d = json.dumps(d)
    bybitOF.input_oi_funding(d)

axis_perpetual_oifunding = {
    "binance_btcusdt" : binance1o,
    "binance_btcusd" : binance2o,
    "okx_btcusdt" : okx1o,
    "okx_btcusd" : okx2o,
    "bybit_btcusdt" : bybitOF,
}
perpAggoi = oiomnifier("btcusd", "perpetual", axis_perpetual_oifunding)
perpAggoi.merge_snapshots()

from flow import indicatorflow
from synthesis import indomnifier

binance_btcusd_gta_data = json.load(open("data/binance_btcusd_perpetual_GTA.json"))
binance_btcusdt_gta_data = json.load(open("data/binance_btcusdt_perpetual_GTA.json"))
okx_btcusdt_gta_data = json.load(open("data/okx_btcusd_perpetual_GTA.json"))
bybit_btcusdt_gta_data = json.load(open("data/bybit_btcusdt_perpetual_GTA.json"))

binance_gta = indicatorflow("btcusdt", "binance", "perpetual", "global_traders_accounts", lookups.binance_GTA_TTA_TTP_lookup)
binance2_gta = indicatorflow("btcusd", "binance", "perpetual", "global_traders_accounts", lookups.binance_GTA_TTA_TTP_lookup)
bybit_gta = indicatorflow("btcusdt", "bybit", "perpetual", "global_traders_accounts", lookups.bybit_GTA_lookup)
okx_gta = indicatorflow("btcALL", "okx", "perpetual", "global_traders_accounts", lookups.okx_GTA_lookup)

for index, d in enumerate(binance_btcusd_gta_data):
    d = json.dumps(d)
    binance_gta.input_binance_gta_tta_ttp(d)
    break

for index, d in enumerate(binance_btcusdt_gta_data):
    d = json.dumps(d)
    binance2_gta.input_binance_gta_tta_ttp(d)
    break

for index, d in enumerate(bybit_btcusdt_gta_data):
    d = json.dumps(d)
    bybit_gta.input_bybit_gta(d)
    break

for index, d in enumerate(okx_btcusdt_gta_data):
    d = json.dumps(d)
    okx_gta.input_okx_gta(d)
    break

axisgta = {
    "binance_usdt" : binance_gta,
    "binance_usd" : binance2_gta,
    "okx_btc_all" : okx_gta, 
    "bybit_usdt" : bybit_gta,
}

gta = indomnifier("btc", "perpetual", "global_traders_accounts", axis=axisgta)
data = perpAggoi.retrive_data("OIs_per_instrument")
gta.inputOI(data)
gta.inputgta()

c:\coding/TradeStreamEngine/StreamEngine\flow.py:318: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  self.snapshot[col] = self.snapshot[col].replace(0, pd.NA).bfill()
c:\coding/TradeStreamEngine/StreamEngine\flow.py:318: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  self.snapshot[col] = self.snapshot[col].replace(0, pd.NA).bfill()
c:\coding/TradeStreamEngine/StreamEngine\flow.py:318: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To

[2.4674, 1.2558, 0.95, 1.1281123643328368] [78688.803, 19616.31011635956, 59315.1417558854]


c:\coding/TradeStreamEngine/StreamEngine\flow.py:318: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  self.snapshot[col] = self.snapshot[col].replace(0, pd.NA).bfill()


# Options Aggregate

In [5]:
from flow import oiflowOption
from synthesis import OOImerger

pranges = np.array([0.0, 1.0, 2.0, 5.0, 10.0])
expiry_windows = np.array([0.0, 1.0, 3.0, 7.0])
boi = oiflowOption("bybit", "btc", "opitons",  
                   pranges,   
                   expiry_windows,
                   lookups.bybit_option_oi_lookup)
ooi = oiflowOption("okx", "btc", "opitons",  
                   pranges,   
                   expiry_windows,
                   lookups.okx_option_oi_lookup)
doi = oiflowOption("deribit", "btc", "opitons",  
                   pranges,   
                   expiry_windows,
                   lookups.deribit_option_oi_lookup)

dt_bybit = json.load(open("data/bybit_btcusdt_option_OI.json"))
dt_okx = json.load(open("data/okx_btc_option_OI.json"))
dt_deribit = json.load(open("data/deribit_btcusd_option_OI.json"))


for index, d in enumerate(dt_bybit):
    d = json.dumps(d)
    boi.input_oi(d)
    break

for index, d in enumerate(dt_okx):
    d = json.dumps(d)
    ooi.input_oi(d)
    break

for index, d in enumerate(dt_deribit):
    d = json.dumps(d)
    doi.input_oi(d)
    break

axisOOI = {
    "okx" : ooi,
    "deribit" : doi,
    "bybit" : boi
}

ooi = OOImerger("btcusdt", "options", expiry_windows, pranges, axisOOI)
ooi.mergeoi()
ooi.retrive_data('puts', "0", "5.0")

126.64999999999998

dict_keys(['0', '0_1', '1_3', '7', '3_7'])

In [25]:
full_call = pd.DataFrame(dtype="float64")
full_put = pd.DataFrame(dtype="float64")

for dfsexch in axisOOI.keys():
    for key_call, key_put in zip(axisOOI[0].df_call.keys(), axisOOI[0].df_put.keys()):
        if full_call.empty:
            full_call = axisOOI[dfsexch]


,-5.0,-2.0,-1.0,0.0,1.0,2.0,5.0,10.0
timestamp,,,,,,,,
2024-01-30 09:00:00,887.64,313.83,2.89,267.3,360.61,484.8,244.02,606.31


# Books Adjustments

In [2]:
from flow import tradesflow, booksflow
from synthesis import tradesmerger, booksmerger, booksadjustments

data_binance_btcusdt_trades = json.load(open("data/binance_btcusdt_spot_trades.json"))
data_binance_btcfdusd_trades = json.load(open("data/binance_btcfdusd_spot_trades.json"))
data_okx_btcusdt_trades = json.load(open("data/okx_btcusdt_spot_trades.json"))
data_bybit_btcusdt_trades = json.load(open("data/bybit_btcusdt_spot_trades.json"))
data_coinbase_btcusd_trades = json.load(open("data/coinbase_btcusd_spot_trades.json"))

b1ts  = tradesflow('binance', 'btc_usdt', 'spot', 20, lookups.binance_trades_lookup)
b2ts  = tradesflow('binance', 'btc_fdusd', 'spot', 20, lookups.binance_trades_lookup)
ots  = tradesflow('okx', 'btc_usdt', 'spot', 20, lookups.okx_trades_lookup)
bbts  = tradesflow('bybit', 'btc_usdt', 'spot', 20, lookups.bybit_trades_lookup)
cts  = tradesflow('coinbase', 'btc_usd', 'spot', 20, lookups.coinbase_trades_lookup)

for index, d in enumerate(data_binance_btcusdt_trades):
    d = json.dumps(d)
    b1ts.input_trades(d)
for index, d in enumerate(data_binance_btcfdusd_trades):
    d = json.dumps(d)
    b2ts.input_trades(d)
for index, d in enumerate(data_okx_btcusdt_trades):
    d = json.dumps(d)
    ots.input_trades(d)
for index, d in enumerate(data_bybit_btcusdt_trades):
    d = json.dumps(d)
    bbts.input_trades(d)
for index, d in enumerate(data_coinbase_btcusd_trades):
    d = json.dumps(d)
    cts.input_trades(d)

axis_spot_trades = {
    "binanceusdt" : b1ts,
    "binancefdusd" : b2ts,
    "okxusdt" : ots,
    "bybitusdt" : bbts,
    "coinbaseusd" : cts,
}
spotAggTrades = tradesmerger("btcusd", "spot", axis_spot_trades)
spotAggTrades.merge_snapshots()


data_binance_btcusdt = json.load(open("data/binance_btcusdt_spot_depth.json"))
data_binance_btcfdusd = json.load(open("data/binance_btcfdusd_spot_depth.json"))
data_okx_btcusdt = json.load(open("data/okx_btcusdt_spot_depth.json"))
data_bybit_btcusdt = json.load(open("data/bybit_btcusdt_spot_depth.json"))
data_coinbase_btcusd = json.load(open("data/coinbase_btcusd_spot_depth.json"))

bbt  = booksflow('binance', 'btc_usdt', 'spot', 20, lookups.binance_depth_lookup)
bbf  = booksflow('binance', 'btc_fdusd', 'spot', 20, lookups.binance_depth_lookup)
obt  = booksflow('okx', 'btc_usdt', 'spot', 20, lookups.okx_depth_lookup)
bybt  = booksflow('bybit', 'btc_usdt', 'spot', 20, lookups.bybit_depth_lookup)
cu  = booksflow('coinbase', 'btc_usd', 'spot', 20, lookups.coinbase_depth_lookup)

for index, d in enumerate(data_binance_btcusdt):
    d = json.dumps(d)
    bbt.update_books(d)
for index, d in enumerate(data_binance_btcfdusd):
    d = json.dumps(d)
    bbf.update_books(d)
for index, d in enumerate(data_okx_btcusdt):
    d = json.dumps(d)
    obt.update_books(d)
for index, d in enumerate(data_bybit_btcusdt):
    d = json.dumps(d)
    bybt.update_books(d)
for index, d in enumerate(data_coinbase_btcusd):
    d = json.dumps(d)
    cu.update_books(d)

axis_spot = {
    "binanceusdt" : bbt,
    "binancefdusd" : bbf,
    "okxusdt" : obt,
    "bybitusdt" : bybt,
    "coinbaseusd" : cu,
}

spotAggDepth = booksmerger("btcusd", "spot", axis_spot)
spotAggDepth.merge_snapshots()

axis_adjOrders = {
    "books" : spotAggDepth,
    "trades" : spotAggTrades
}


c:\coding/TradeStreamEngine/StreamEngine\flow.py:197: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  self.snapshot_buys['price'] = self.snapshot_buys['price'].replace(0, pd.NA).bfill()
c:\coding/TradeStreamEngine/StreamEngine\flow.py:199: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  self.snapshot_sells['price'] = self.snapshot_sells['price'].replace(0, pd.NA).bfill()
c:\coding/TradeStreamEngine/StreamEngine\flow.py:196: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result

# Liquidations aggregate

In [2]:
from flow import liquidationsflow
from synthesis import lomnifier

data_binance_btcusdt_liquidations = json.load(open("data/binance_btcusdt_perpetual_liquidations.json"))
data_binance_btcusd_liquidations = json.load(open("data/binance_btcusd_perpetual_liquidations.json"))
data_okx_btcusdt_liquidations = json.load(open("data/okx_btcusdt_perpetual_liquidations.json"))
data_okx_btcusd_liquidations = json.load(open("data/okx_btcusdt_perpetual_liquidations.json"))
data_bybit_btcusdt_liquidations = json.load(open("data/bybit_btcusdt_perpetual_liquidations.json"))

binance_1_liquidations  = liquidationsflow('binance', 'btc_usdt', 'perpetual', 20, lookups.binance_liquidations_lookup)
binance_2_liquidations  = liquidationsflow('binance', 'btv_usd', 'perpetual', 20, lookups.binance_liquidations_lookup)
okx_1_liquidations  = liquidationsflow('okx', 'btc_usdt', 'perpetual', 20, lookups.okx_liquidations_lookup)
okx_2_liquidations  = liquidationsflow('okx', 'btc_usd', 'perpetual', 20, lookups.okx_liquidations_lookup)
bybit_1_liquidations  = liquidationsflow('bybit', 'btc_usdt', 'perpetual', 20, lookups.bybit_liquidations_lookup)

for index, d in enumerate(data_binance_btcusdt_liquidations):
    d = json.dumps(d)
    binance_1_liquidations.input_liquidations(d)
for index, d in enumerate(data_binance_btcusd_liquidations):
    d = json.dumps(d)
    binance_2_liquidations.input_liquidations(d)
for index, d in enumerate(data_okx_btcusdt_liquidations):
    d = json.dumps(d)
    okx_1_liquidations.input_liquidations(d)
for index, d in enumerate(data_okx_btcusd_liquidations):
    d = json.dumps(d)
    okx_2_liquidations.input_liquidations(d)
for index, d in enumerate(data_bybit_btcusdt_liquidations):
    d = json.dumps(d)
    bybit_1_liquidations.input_liquidations(d)

axis_liquidations = {
    "binance_1" : binance_1_liquidations,
    "binance_2" : binance_2_liquidations,
    "okx_1" : okx_1_liquidations,
    "okx_2" : okx_2_liquidations,
    "bybit" : bybit_1_liquidations,
}
spotAggLiquidations = lomnifier("btcusd", "spot", axis_liquidations)
spotAggLiquidations.merge_snapshots()

# Trades Merger

In [2]:
from flow import tradesflow
from synthesis import tradesmerger

data_binance_btcusdt_trades = json.load(open("data/binance_btcusdt_spot_trades.json"))
data_binance_btcfdusd_trades = json.load(open("data/binance_btcfdusd_spot_trades.json"))
data_okx_btcusdt_trades = json.load(open("data/okx_btcusdt_spot_trades.json"))
data_bybit_btcusdt_trades = json.load(open("data/bybit_btcusdt_spot_trades.json"))
data_coinbase_btcusd_trades = json.load(open("data/coinbase_btcusd_spot_trades.json"))

b1ts  = tradesflow('binance', 'btc_usdt', 'spot', 20, lookups.binance_trades_lookup)
b2ts  = tradesflow('binance', 'btc_fdusd', 'spot', 20, lookups.binance_trades_lookup)
ots  = tradesflow('okx', 'btc_usdt', 'spot', 20, lookups.okx_trades_lookup)
bbts  = tradesflow('bybit', 'btc_usdt', 'spot', 20, lookups.bybit_trades_lookup)
cts  = tradesflow('coinbase', 'btc_usd', 'spot', 20, lookups.coinbase_trades_lookup)

for index, d in enumerate(data_binance_btcusdt_trades):
    d = json.dumps(d)
    b1ts.input_trades(d)
for index, d in enumerate(data_binance_btcfdusd_trades):
    d = json.dumps(d)
    b2ts.input_trades(d)
for index, d in enumerate(data_okx_btcusdt_trades):
    d = json.dumps(d)
    ots.input_trades(d)
for index, d in enumerate(data_bybit_btcusdt_trades):
    d = json.dumps(d)
    bbts.input_trades(d)
for index, d in enumerate(data_coinbase_btcusd_trades):
    d = json.dumps(d)
    cts.input_trades(d)

axis_spot_trades = {
    "binanceusdt" : b1ts,
    "binancefdusd" : b2ts,
    "okxusdt" : ots,
    "bybitusdt" : bbts,
    "coinbaseusd" : cts,
}
spotAggTrades = tradesmerger("btcusd", "spot", axis_spot_trades)
spotAggTrades.merge_snapshots()

c:\coding/TradeStreamEngine/StreamEngine\flow.py:197: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  self.snapshot_buys['price'] = self.snapshot_buys['price'].replace(0, pd.NA).bfill()
c:\coding/TradeStreamEngine/StreamEngine\flow.py:199: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  self.snapshot_sells['price'] = self.snapshot_sells['price'].replace(0, pd.NA).bfill()
c:\coding/TradeStreamEngine/StreamEngine\flow.py:196: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result

## OI_Funging Aggregate


In [12]:
from flow import oiFundingflow
from synthesis import oiomnifier

data_binance_btcusdt_OI = json.load(open("data/binance_btcusdt_perpetual_OI.json"))
data_binance_btcusdt_funding = json.load(open("data/binance_btcusdt_perpetual_fundingRate.json"))
data_binance_btcusd_OI = json.load(open("data/binance_btcusd_perpetual_OI.json"))
data_binance_btcusd_funding = json.load(open("data/binance_btcusd_perpetual_fundingRate.json"))
data_okx_btcusdt_OI = json.load(open("data/okx_btcusdt_perpetual_OI.json"))
data_okx_btcusdt_funding = json.load(open("data/okx_btcusdt_perpetual_fundingRate.json"))
data_okx_btcusd_OI = json.load(open("data/okx_btcusd_perpetual_OI.json"))
data_okx_btcusd_funding = json.load(open("data/okx_btcusd_perpetual_fundingRate.json"))
data_bybit_btcusdt_oifunding = json.load(open("data/bybit_btcusdt_perpetual_fundingRate_OI.json"))

binance1o  = oiFundingflow('binance', 'btc_usdt', 'perpetual', 20, lookups.binance_OI_lookup, lookups.binance_funding_lookup)
binance2o  = oiFundingflow('binance', 'btc_usd', 'perpetual', 20, lookups.binance_OI_lookup, lookups.binance_funding_lookup)
okx1o  = oiFundingflow('okx', 'btc_usdt', 'perpetual', 20, lookups.okx_OI_lookup, lookups.okx_funding_lookup)
okx2o  = oiFundingflow('okx', 'btc_usd', 'perpetual', 20, lookups.okx_OI_lookup, lookups.okx_funding_lookup)
bybitOF  = oiFundingflow('bybit', 'btc_usdt', 'perpetual', 20, lookups.bybit_OI_funding_lookup)


for index, d in enumerate(data_binance_btcusdt_funding):
    d = json.dumps(d)
    binance1o.input_funding(d)
for index, d in enumerate(data_binance_btcusdt_OI):
    d = json.dumps(d)
    binance1o.input_oi(d)

for index, d in enumerate(data_binance_btcusd_funding):
    d = json.dumps(d)
    binance2o.input_funding(d)
for index, d in enumerate(data_binance_btcusd_OI):
    d = json.dumps(d)
    binance2o.input_oi(d)

for index, d in enumerate(data_okx_btcusdt_funding):
    d = json.dumps(d)
    okx1o.input_funding(d)
for index, d in enumerate(data_okx_btcusdt_OI):
    d = json.dumps(d)
    okx1o.input_oi(d)

for index, d in enumerate(data_okx_btcusd_funding):
    d = json.dumps(d)
    okx2o.input_funding(d)
for index, d in enumerate(data_okx_btcusd_OI):
    d = json.dumps(d)
    okx2o.input_oi(d)

for index, d in enumerate(data_bybit_btcusdt_oifunding):
    d = json.dumps(d)
    bybitOF.input_oi_funding(d)

axis_perpetual_oifunding = {
    "binance_btcusdt" : binance1o,
    "binance_btcusd" : binance2o,
    "okx_btcusdt" : okx1o,
    "okx_btcusd" : okx2o,
    "bybit_btcusdt" : bybitOF,
}
perpAggoi = oiomnifier("btcusd", "perpetual", axis_perpetual_oifunding)
perpAggoi.merge_snapshots()

perpAggoi.snapshot

c:\coding/TradeStreamEngine/StreamEngine\flow.py:318: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  self.snapshot[col] = self.snapshot[col].replace(0, pd.NA).bfill()
c:\coding/TradeStreamEngine/StreamEngine\flow.py:318: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  self.snapshot[col] = self.snapshot[col].replace(0, pd.NA).bfill()
c:\coding/TradeStreamEngine/StreamEngine\flow.py:318: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To

# Trades Aggregate

In [4]:
from flow import tradesflow
from synthesis import tradesmerger

data_binance_btcusdt_trades = json.load(open("data/binance_btcusdt_spot_trades.json"))
data_binance_btcfdusd_trades = json.load(open("data/binance_btcfdusd_spot_trades.json"))
data_okx_btcusdt_trades = json.load(open("data/okx_btcusdt_spot_trades.json"))
data_bybit_btcusdt_trades = json.load(open("data/bybit_btcusdt_spot_trades.json"))
data_coinbase_btcusd_trades = json.load(open("data/coinbase_btcusd_spot_trades.json"))

b1ts  = tradesflow('binance', 'btc_usdt', 'spot', 20, lookups.binance_trades_lookup)
b2ts  = tradesflow('binance', 'btc_fdusd', 'spot', 20, lookups.binance_trades_lookup)
ots  = tradesflow('okx', 'btc_usdt', 'spot', 20, lookups.okx_trades_lookup)
bbts  = tradesflow('bybit', 'btc_usdt', 'spot', 20, lookups.bybit_trades_lookup)
cts  = tradesflow('coinbase', 'btc_usd', 'spot', 20, lookups.coinbase_trades_lookup)

for index, d in enumerate(data_binance_btcusdt_trades):
    d = json.dumps(d)
    b1ts.input_trades(d)
for index, d in enumerate(data_binance_btcfdusd_trades):
    d = json.dumps(d)
    b2ts.input_trades(d)
for index, d in enumerate(data_okx_btcusdt_trades):
    d = json.dumps(d)
    ots.input_trades(d)
for index, d in enumerate(data_bybit_btcusdt_trades):
    d = json.dumps(d)
    bbts.input_trades(d)
for index, d in enumerate(data_coinbase_btcusd_trades):
    d = json.dumps(d)
    cts.input_trades(d)

c:\coding/TradeStreamEngine/StreamEngine\flow.py:172: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  self.snapshot_buys['price'] = self.snapshot_buys['price'].replace(0, pd.NA).bfill()
c:\coding/TradeStreamEngine/StreamEngine\flow.py:174: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  self.snapshot_sells['price'] = self.snapshot_sells['price'].replace(0, pd.NA).bfill()
c:\coding/TradeStreamEngine/StreamEngine\flow.py:171: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result

In [5]:
axis_spot_trades = {
    "binanceusdt" : b1ts,
    "binancefdusd" : b2ts,
    "okxusdt" : ots,
    "bybitusdt" : bbts,
    "coinbaseusd" : cts,
}
spotAggTrades = tradesmerger("btcusd", "spot", axis_spot_trades)
spotAggTrades.merge_snapshots()

KeyError: "['price'] not found in axis"

In [10]:
from flow import tradesflow
from synthesis import tradesmerger

data_binance_btcusdt_trades = json.load(open("data/binance_btcusdt_perpetual_trades.json"))
data_binance_btcfdusd_trades = json.load(open("data/binance_btcusd_perpetual_trades.json"))
data_okx_btcusdt_trades = json.load(open("data/okx_btcusdt_perpetual_trades.json"))
data_bybit_btcusdt_trades = json.load(open("data/bybit_btcusdt_perpetual_trades.json"))
data_coinbase_btcusd_trades = json.load(open("data/okx_btcusd_perpetual_trades.json"))

b1tp  = tradesflow('binance', 'btc_usdt', 'perpetual', 20, lookups.binance_trades_lookup)
b2tp  = tradesflow('binance', 'btc_fdusd', 'perpetual', 20, lookups.binance_trades_lookup)
otp  = tradesflow('okx', 'btc_usdt', 'perpetual', 20, lookups.okx_trades_lookup)
bbtp  = tradesflow('bybit', 'btc_usdt', 'perpetual', 20, lookups.bybit_trades_lookup)
ctp  = tradesflow('coinbase', 'btc_usd', 'perpetual', 20, lookups.okx_trades_lookup)

for index, d in enumerate(data_binance_btcusdt_trades):
    d = json.dumps(d)
    b1tp.input_trades(d)
for index, d in enumerate(data_binance_btcfdusd_trades):
    d = json.dumps(d)
    b2tp.input_trades(d)
for index, d in enumerate(data_okx_btcusdt_trades):
    d = json.dumps(d)
    otp.input_trades(d)
for index, d in enumerate(data_bybit_btcusdt_trades):
    d = json.dumps(d)
    bbtp.input_trades(d)
for index, d in enumerate(data_coinbase_btcusd_trades):
    d = json.dumps(d)
    ctp.input_trades(d)

In [5]:
axis_perpetual_trades = {
    "binance1" : b1tp,
    "binance2" : b2tp,
    "okx1" : otp,
    "bybit" : bbtp,
    "okx2" : ctp,
}
perpetualAggTrades = tradesmerger("btcusd", "perpetual", axis_perpetual_trades)
perpetualAggTrades.merge_snapshots()

NameError: name 'b1tp' is not defined

In [4]:
perpetualAggTrades.snapshotO

NameError: name 'perpetualAggTrades' is not defined

# Books Aggregate

In [15]:
from synthesis import booksmerger
from flow import booksflow

In [3]:
data_binance_btcusdt = json.load(open("data/binance_btcusdt_spot_depth.json"))
data_binance_btcfdusd = json.load(open("data/binance_btcfdusd_spot_depth.json"))
data_okx_btcusdt = json.load(open("data/okx_btcusdt_spot_depth.json"))
data_bybit_btcusdt = json.load(open("data/bybit_btcusdt_spot_depth.json"))
data_coinbase_btcusd = json.load(open("data/coinbase_btcusd_spot_depth.json"))

bbt  = booksflow('binance', 'btc_usdt', 'spot', 20, lookups.binance_depth_lookup)
bbf  = booksflow('binance', 'btc_fdusd', 'spot', 20, lookups.binance_depth_lookup)
obt  = booksflow('okx', 'btc_usdt', 'spot', 20, lookups.okx_depth_lookup)
bybt  = booksflow('bybit', 'btc_usdt', 'spot', 20, lookups.bybit_depth_lookup)
cu  = booksflow('coinbase', 'btc_usd', 'spot', 20, lookups.coinbase_depth_lookup)

for index, d in enumerate(data_binance_btcusdt):
    d = json.dumps(d)
    bbt.update_books(d)
for index, d in enumerate(data_binance_btcfdusd):
    d = json.dumps(d)
    bbf.update_books(d)
for index, d in enumerate(data_okx_btcusdt):
    d = json.dumps(d)
    obt.update_books(d)
for index, d in enumerate(data_bybit_btcusdt):
    d = json.dumps(d)
    bybt.update_books(d)
for index, d in enumerate(data_coinbase_btcusd):
    d = json.dumps(d)
    cu.update_books(d)

In [21]:
data_binance_btcusdt_perp = json.load(open("data/binance_btcusdt_perpetual_depth.json"))
data_binance_btcusd_perp = json.load(open("data/binance_btcusd_perpetual_depth.json"))
data_okx_btcusdt_perp = json.load(open("data/okx_btcusdt_perpetual_depth.json"))
data_bybit_btcusdt_perp = json.load(open("data/bybit_btcusdt_perpetual_depth.json"))
data_okx_btcusd_perp = json.load(open("data/okx_btcusd_perpetual_depth.json"))

binance_1  = booksflow('binance', 'btc_usdt', 'perpetual', 20, lookups.binance_depth_lookup)
binance_2  = booksflow('binance', 'btc_usd', 'perpetual', 20, lookups.binance_depth_lookup)
okx_1  = booksflow('okx', 'btc_usdt', 'perpetual', 20, lookups.okx_depth_lookup)
bybit  = booksflow('bybit', 'btc_usdt', 'perpetual', 20, lookups.bybit_depth_lookup)
okx_2  = booksflow('okx', 'btc_usd', 'perpetual', 20, lookups.okx_depth_lookup)

for index, d in enumerate(data_binance_btcusdt_perp):
    d = json.dumps(d)
    binance_1.update_books(d)
for index, d in enumerate(data_binance_btcusd_perp):
    d = json.dumps(d)
    binance_2.update_books(d)
for index, d in enumerate(data_okx_btcusdt_perp):
    d = json.dumps(d)
    okx_1.update_books(d)
for index, d in enumerate(data_bybit_btcusdt_perp):
    d = json.dumps(d)
    bybit.update_books(d)
for index, d in enumerate(data_okx_btcusd_perp):
    d = json.dumps(d)
    okx_2.update_books(d)

In [23]:
axis_perp = {
    "binanceusdt" : binance_1,
    "binanceusd" : binance_2,
    "okxusdt" : okx_1,
    "okxusd" : okx_2,
    "bybitusdt" : bybit,
}
perpAggDepth = booksmerger("btcusd", "spot", axis_perp)
perpAggDepth.merge_snapshots()

perpAggDepth.snapshot

,41680.0,41900.0,41960.0,42000.0,42080.0,42120.0,42320.0,42340.0,42380.0,42400.0,...,44560.0,44640.0,44680.0,44720.0,44740.0,44880.0,44940.0,45000.0,45120.0,45160.0
0,0.096706,3.859375,3.859375,0.373047,12.249512,4.36084,24.324921,14.597656,44.816406,3.244141,...,0.004604,13.363979,4.246094,4.109375,0.672363,4.148438,0.004612,0.009224,0.004612,0.009209
1,0.096706,3.859375,3.859375,0.373047,12.249512,4.36084,24.324921,14.597656,44.816406,3.244141,...,0.004604,13.363979,4.246094,4.109375,0.672363,4.148438,0.004612,0.009224,0.004612,0.009209
2,0.096706,3.859375,3.859375,0.373047,12.249512,4.36084,24.324921,14.597656,44.816406,3.244141,...,0.004604,13.363979,4.246094,4.109375,0.672363,4.148438,0.004612,0.009224,0.004612,0.009209
3,0.096706,3.859375,3.859375,0.373047,12.249512,4.36084,24.324921,14.597656,44.816406,3.244141,...,0.004604,13.363979,4.246094,4.109375,0.672363,4.148438,0.004612,0.009224,0.004612,0.009209
4,0.096706,3.859375,3.859375,0.373047,12.249512,4.36084,24.324921,14.597656,44.816406,3.244141,...,0.004604,13.363979,4.246094,4.109375,0.672363,4.148438,0.004612,0.009224,0.004612,0.009209
5,0.096706,3.859375,3.859375,0.373047,12.249512,4.36084,24.324921,14.597656,44.816406,3.244141,...,0.004604,13.363979,4.246094,4.109375,0.672363,4.148438,0.004612,0.009224,0.004612,0.009209
6,0.096706,3.859375,3.859375,0.373047,12.249512,4.36084,24.324921,14.597656,44.816406,3.244141,...,0.004604,13.363979,4.246094,4.109375,0.672363,4.148438,0.004612,0.009224,0.004612,0.009209
7,0.096706,3.859375,3.859375,0.373047,12.249512,4.36084,24.324921,14.597656,44.816406,3.244141,...,0.004604,13.363979,4.246094,4.109375,0.672363,4.148438,0.004612,0.009224,0.004612,0.009209
8,0.096706,3.859375,3.859375,0.373047,12.249512,4.36084,24.324921,14.597656,44.816406,3.244141,...,0.004604,13.363979,4.246094,4.109375,0.672363,4.148438,0.004612,0.009224,0.004612,0.009209
9,0.096706,3.859375,3.859375,0.373047,12.249512,4.36084,24.324921,14.597656,44.816406,3.244141,...,0.004604,13.363979,4.246094,4.109375,0.672363,4.148438,0.004612,0.009224,0.004612,0.009209


In [17]:
axis_spot = {
    "binanceusdt" : bbt,
    "binancefdusd" : bbf,
    "okxusdt" : obt,
    "bybitusdt" : bybt,
    "coinbaseusd" : cu,
}

spotAggDepth = booksmerger("btcusd", "spot", axis_spot)
spotAggDepth.merge_snapshots()

NameError: name 'bbt' is not defined